In [1]:
import IPython.display as disp

# Quickstart Tutorial: Working with the symbolic transformed payoff matrix

The purpose of this tutorial is to show how the transformed payoff matrix can shed light on the evolutionary dynamics of a game under homophily. For this purpose, we will use a small 3-player public goods game with coordinated cooperation as our example. 

## 1. Original and transformed payoff matrices

We consider an example where the benefits returned from the public good have a sigmoid-like relationship with the number of contributors. 

Because the number of players is small ($n=3$), rather that specifying the benefits function in terms of a full sigmoid function, we instead define the benefit returned at each contribution level in terms of fixed parameters. Let $B(k)$ be the benefit returned from the public good when $k$ contribute, then we define

$$
[B(0), B(1), B(2), B(3)] = [0, \beta, b-\beta, b].
$$

We specify the coordinator quorum and contribution level $\tau = 2$. As in the general sigmoid, the cost of contributing is $c$, and the cognitive cost to Coordinating Cooperators and Liars for the capacity to understand the game and coordinate is $\varepsilon$.

We will be working with the Python library for symbolic mathematics `SymPy`, and the functions for automatically generating the payoff matrices are included in the repository.

In [2]:
import sympy as sp

import sys
sys.path.append('../functions/')

from symbolic_transformed import create_payoff_matrix
from symbolic_transformed import create_transformed_payoff_matrix
from symbolic_transformed import calc_switching_gains

The first step is to specify our payoff function, which the function `create_payoff_matrix()` will use to generate the payoff matrix $A$. The function documentation specifies that the payoff function should have the form `pi(gamma_focal, gamma_nonfocal)`. 

The input `gamma_focal` is called $\boldsymbol{\gamma}_0$ in the main text. It is a symbolic matrix of length `m` with a `1` in the `i`-th position indicating that the focal individual plays strategy `i`.

The input `gamma_nonfocal` is called $\boldsymbol{\gamma}_{-0}$ in the main text. It is a symbolic matrix of length `m` where the value of each entry `j` is how many individuals among the `n-1` nonfocal group members play strategy `j`.

The function `pi(gamma_focal, gamma_nonfocal)` will return a symbolic expression.

In [3]:
def pi(gamma_f, gamma_n):

    # fixed parameters that define the game
    # ---

    # defined
    n = 3   # group size
    tau = 2 # threshold

    # symbolic - benefit, contrib cost, cognitive cost
    b, c, eps, beta = sp.symbols('b, c, epsilon, beta')

    # benefit function, PG returns b if no. contributors >= tau
    B = lambda k: [0, beta, b-beta, b][k]

    # strategies
    strat_names = ['D', 'C', 'L', 'U']
    m = len(strat_names)   # number of strategies


    # calculate payoff to focal who plays gamma_f
    # given other members play gamma_n

    # whole group distribution
    gamma = gamma_f + gamma_n

    # count the number of each strategy in the group
    countD, countC, countL, countU = gamma

    # identify the focal strategy
    focal_strat = [strat for strat, v in zip(['D', 'C', 'L', 'U'], gamma_f) if v == 1][0]

    # payoff calculation depends on if lottery quorum is met

    if countC + countL < tau: # if the lottery quorum is not met

        k = countU                                      # only U will contribute
        benefit = B(k) # benefit from PGG

        # payoff returned depending on the strategy of the focal player
        if focal_strat == 'U':

            payoff_total = benefit - c

        elif focal_strat == 'D':

            payoff_total = benefit

        else: # focal_strat == C or L

            payoff_total = benefit - eps

    else: # lottery quorum is met

        # probability that j Coordinating Cooperators will be designated as contributors by the lottery
        denom = sp.binomial(countC + countL, tau)
        PjV = [sp.binomial(countC, j)*sp.binomial(countL, tau-j)/denom for j in range(0, tau+1)]

        # the benefit returned for each j
        benefitjV = [B(j+countU) for j in range(0, tau+1)]

        # payoff returned depending on the strategy of the focal player
        if focal_strat == 'U':

            payoff_total = sum(Pj*benefitj for Pj, benefitj in zip(PjV, benefitjV)) - c

        elif focal_strat == 'D':

            payoff_total = sum(Pj*benefitj for Pj, benefitj in zip(PjV, benefitjV))

        elif focal_strat == 'C':

            payoff_total = sum(PjV[j]*(benefitjV[j] - c*j/countC) for j in range(0, tau+1)) - eps

        else: # focal_strat == 'L':

            payoff_total = sum(Pj*benefitj for Pj, benefitj in zip(PjV, benefitjV)) - eps


    return payoff_total



The original matrix is created as follows:

In [4]:
n = 3 # number of individuals in the group
m = 4 # number of strategies

A = create_payoff_matrix(n, m, pi)
A

[[[0, 0, 0, beta], [0, b - beta, beta, beta], [0, beta, 0, beta], [beta, beta, beta, b - beta]], [[-epsilon, b - beta - c - epsilon, beta - c - epsilon, beta - epsilon], [b - beta - c - epsilon, b - beta - 2*c/3 - epsilon, b/3 + beta/3 - 2*c/3 - epsilon, b - c - epsilon], [beta - c - epsilon, b/3 + beta/3 - 2*c/3 - epsilon, 2*beta/3 - 2*c/3 - epsilon, b - beta - c - epsilon], [beta - epsilon, b - c - epsilon, b - beta - c - epsilon, b - beta - epsilon]], [[-epsilon, beta - epsilon, -epsilon, beta - epsilon], [beta - epsilon, b/3 + beta/3 - epsilon, 2*beta/3 - epsilon, b - beta - epsilon], [-epsilon, 2*beta/3 - epsilon, -epsilon, beta - epsilon], [beta - epsilon, b - beta - epsilon, beta - epsilon, b - beta - epsilon]], [[beta - c, beta - c, beta - c, b - beta - c], [beta - c, b - c, b - beta - c, b - beta - c], [beta - c, b - beta - c, beta - c, b - beta - c], [b - beta - c, b - beta - c, b - beta - c, b - c]]]

The transformed payoff matrix is created from $A$ as follows:

In [5]:
B = create_transformed_payoff_matrix(A)
B

[[[0, F_2*(b - beta)/6, 0, F_1*beta + F_2*beta/3 + F_2*(b - beta)/6], [F_2*(b - beta)/6, F_1*(b - beta) + F_2*(b - beta)/3, F_1*beta + F_2*(b - beta)/6, F_1*beta + F_2*beta/3 + F_2*(b - beta)/3], [0, F_1*beta + F_2*(b - beta)/6, 0, F_1*beta + F_2*beta/3 + F_2*(b - beta)/6], [F_1*beta + F_2*beta/3 + F_2*(b - beta)/6, F_1*beta + F_2*beta/3 + F_2*(b - beta)/3, F_1*beta + F_2*beta/3 + F_2*(b - beta)/6, F_1*(b - beta) + 2*F_2*beta/3 + F_2*(b - beta)/3]], [[-F_1*epsilon - F_2*epsilon/3 + 2*F_2*(b - beta - c - epsilon)/3 + F_3*(b - beta - 2*c/3 - epsilon), F_1*(b - beta - c - epsilon) - F_2*epsilon/6 + F_2*(b - beta - c - epsilon)/3 + F_2*(b - beta - 2*c/3 - epsilon)/2 + F_3*(b - beta - 2*c/3 - epsilon), F_1*(beta - c - epsilon) - F_2*epsilon/6 + F_2*(2*beta/3 - 2*c/3 - epsilon)/6 + F_2*(b/3 + beta/3 - 2*c/3 - epsilon)/3 + F_2*(b - beta - c - epsilon)/3 + F_3*(b - beta - 2*c/3 - epsilon), F_1*(beta - epsilon) - F_2*epsilon/6 + F_2*(b - beta - epsilon)/6 + F_2*(b - c - epsilon)/3 + F_2*(b - be

Note the subscripts used for the partition-structure probabilities $F$ refer to an indexation of the partitions of $n$, which is the ordering used in the SI about relatedness coefficients, and is the ordering in which they returned by the `utilities` function `partitionInteger()`. 

In this example, $F_1 \equiv F_{[1,1,1]}$, $F_2 \equiv F_{[2,1]}$, and $F_3 \equiv F_{[3]}$.

In [6]:
from utilities import partitionInteger

partnV = list(partitionInteger(n))
partnV

[[1, 1, 1], [1, 2], [3]]

## 2. Switching gains analysis

Taking inspiration from Peña et al. (2014), we can use the transformed payoffs to gain qualitative insights into the game under homophily. Symbolic expressions for the switching gains between pairs of strategies (along with the payoffs) can be generated using the function `calc_switching_gains()`. Below, we will do two small examples to illustrate the kind of analysis that is possible.

### 2.1 Example: Unconditional Cooperators versus Unconditional Defectors

The switching gains to Unconditional Cooperators (strategy 3) against Defectors (strategy 0) in a well-mixed population are:

In [9]:
mix_dk, pi_foc, pi_nonfoc = calc_switching_gains(A, 3, 0)

for k, dk in enumerate(mix_dk):
    
    print(f'switching gain d_{k}')
    disp.display(dk)

switching gain d_0


beta - c

switching gain d_1


b - 2*beta - c

switching gain d_2


beta - c

We are interested in scenarios where the lone contributor faces a social dilemma; therefore, we assume

$$
\beta-c < 0.
$$

In the full sigmoid gain, we focused on the existence of a positive switching gain when there were $\tau$ U-strategists and $n-\tau$ D-strategists, and derived the condition: $B(\tau) - B(\tau-1) - c > 0$. In this example, the analogous condition is

$$
b - 2\beta - c > 0.
$$

In the homophilic population, the switching gains are:

In [10]:
hom_dk, pi_foc, pi_nonfoc = calc_switching_gains(B, 3, 0)

# tidy the dk expressions so the terms are grouped by F_i
num_partns = len(partnV)
hom_dk = [sp.collect(expr, [sp.Symbol('F_'+str(i)) for i in range(1, num_partns+1)]) 
                for expr in hom_dk]
        
for k, dk in enumerate(hom_dk):
    
    print(f'switching gain d_{k}')
    disp.display(dk)

switching gain d_0


F_1*(beta - c) + F_2*(2*b/3 - beta/3 - c) + F_3*(b - c)

switching gain d_1


F_1*(b - 2*beta - c) + F_2*(2*b/3 - beta/3 - c) + F_3*(b - c)

switching gain d_2


F_1*(beta - c) + F_2*(2*b/3 - beta/3 - c) + F_3*(b - c)

The $F_3$ components $b-c > 0$, so under perfect homophily, U dominate.

The $F_2$ component $(2b-\beta-3c)/3$ is positive if the switching gain above is positive and $3\beta - c > 0$. If this is satisfied, the effect of homophily is to shift the gains function uniformly "upwards", thus increasing the possibility for coexistence of C and D and (eventually) dominance of C.

### 2.2 Coordinating Cooperators versus Liars -- Invasion of Liars

The switching gains to Coordinating Cooperators (strategy 1) against Liars (strategy 2) in a well-mixed population are:

In [11]:
mix_dk, pi_foc, pi_nonfoc = calc_switching_gains(A, 1, 2)

for k, dk in enumerate(mix_dk):
    
    print(f'switching gain d_{k}')
    disp.display(dk)


switching gain d_0


2*beta/3 - 2*c/3

switching gain d_1


b/3 - beta/3 - 2*c/3

switching gain d_2


2*b/3 - 4*beta/3 - 2*c/3

The invasion fitness of a B-strategist into an all-A population is equal to 
the negative of A's switching gain $-d_{n-1}$. From the switching gains above, 
we learn that, in a well-mixed population,
the invasion fitness of L into an all-C population is

$$
    \left. \frac{\dot{p}_L}{p_L} \right|_{p_C=1} = \frac{-(2b - 4\beta - 2c)}{3}.
    \label{A:sigsimpler_invL_to_C_mix}
$$


Therefore, if there is a positive switching gain for $\tau$ U-strategists, because 

$$
b - 2\beta - c > 0 \ \implies \frac{2b - 4\beta - 2c}{3} > 0.
$$

then Liars cannot invade a population of all Coordinating Cooperators.

The switching gains to Coordinating Cooperators (strategy 1) against Liars (strategy 2) in a homophilic population are:

In [12]:
hom_dk, pi_foc, pi_nonfoc = calc_switching_gains(B, 1, 2)

# tidy the dk expressions so the terms are grouped by F_i
num_partns = len(partnV)
hom_dk = [sp.collect(expr, [sp.Symbol('F_'+str(i)) for i in range(1, num_partns+1)]) 
                for expr in hom_dk]
        
for k, dk in enumerate(hom_dk):
    
    print(f'switching gain d_{k}')
    disp.display(dk)

switching gain d_0


F_1*(2*beta/3 - 2*c/3) + F_2*(2*b/9 + 4*beta/9 - 2*c/3) + F_3*(b - beta - 2*c/3)

switching gain d_1


F_1*(b/3 - beta/3 - 2*c/3) + F_2*(5*b/9 - 5*beta/9 - 2*c/3) + F_3*(b - beta - 2*c/3)

switching gain d_2


F_1*(2*b/3 - 4*beta/3 - 2*c/3) + F_2*(8*b/9 - 14*beta/9 - 2*c/3) + F_3*(b - beta - 2*c/3)

Here we learn that, when there is some homophily,
the invasion fitness of L into an all-C population is

$$
    \left. \frac{\dot{p}_L}{p_L} \right|_{p_C=1} = 
    - \left[
        F_{[1,1,1]} \underbrace{\left(\frac{2b-4\beta-2c}{3}\right)}_{X}+
        F_{[2,1]} \underbrace{\left(\frac{8b-14\beta-6c}{9}\right)}_{Y}+
        F_{[3]} \underbrace{\left(\frac{3b-3\beta-2c}{3}\right)}_{Z} \right].
$$

First, we can show that, if $b-2\beta-c > 0$, then $Y > 0$ and $X > 0$. Therefore, L cannot invade under homophily. 
(To show, e.g., $Y>0$, let $x$ be some positive value, define $b = 2\beta+c+x$, and substitute into the expression for $Y$.)

Furthermore, if $b-2\beta-c > 0$, then the invasion fitness of L always becomes more negative as homophily increases. As homophily increases, $F_{[2,1]}$ and $F_{[3]}$ increase and $F_{[1,1,1]}$ decreases.
If $b-2\beta-c > 0$, then $Y > X$ and $Z > X$.
Therefore, as homophily increases, the invasion fitness of L into an all-C population decreases.


### 2.3 Outputting results to LaTeX

The module `symbolic_transformed` also has a function `tabulate_switching_gains`, which will format a LaTeX table of the switching gains. 